### DCNN : CNN + Pooling

- 지역 수용 영역 : 지역적인 구조를 인코딩 - 인접한 입력 뉴런의 부분 행렬들을 다음 계층에 있는 한 개의 은닉 뉴런으로 연결
- 풀링 : 맵 출력을 요약, 하나의 특징 맵으로부터 나온 출력에서 공간 인접성을 활용해 부분 행렬 값들을 물리적인 지역에 관련된 의미를 종합적으로 설명하는 출력 값으로 결합

    - max pooling : 관찰한 최대 활성화를 출력
    model.add(MaxPooling2D(pool_size= (2,2))
    - avg pooling :
    
- DCNN의 낮은 계층에서는 색상이나 가장자리 같은 낮은 차원의 특징을 식별.
- 중간계층은 이미지에서 중요한 특징을 추출할 수 있는 능력을 가지고 있다

In [ ]:
from keras import backend as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense, Dropout

from keras.datasets import cifar10
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2018)

In [ ]:
class Simple:
    @staticmethod
    def build(input_shape, classes):
        model = Sequential()
        
        model.add(Conv2D(32, kernel_size=3, padding='same', input_shape=input_shape))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        model.add(Dropout(0.25))
                
        model.add(Flatten())
        
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        
        model.add(Dense(classes))
        model.add(Activation('softmax'))
        return model

In [ ]:
NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1

OPTIMIZER = RMSprop()
VALIDATION_SPLIT = 0.2
IMG_CHANNEL, IMG_ROWS, IMG_COLS = 3, 32, 32
NB_CLASSES = 10
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, IMG_CHANNEL)

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
y_train = np_utils.to_categorical(y_train, NB_CLASSES)
y_test = np_utils.to_categorical(y_test, NB_CLASSES)

In [ ]:
model = Simple.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

history = model.fit(X_train, y_train, \
                   batch_size=BATCH_SIZE, epochs=NB_EPOCH,\
                   verbose=VERBOSE, validation_split=VALIDATION_SPLIT)
score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def draw(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
draw(history)

In [ ]:
class More_Layer:
    @staticmethod
    def build(input_shape, classes):
        model = Sequential()
        
        model.add(Conv2D(32, kernel_size=3, padding='same', input_shape=input_shape))
        model.add(Activation('relu'))
        
        model.add(Conv2D(32, kernel_size=3, padding='same'))
        model.add(Activation('relu'))
        
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(64, kernel_size=3, padding='same'))
        model.add(Activation('relu'))

        model.add(Conv2D(64, kernel_size=3, padding='same'))
        model.add(Activation('relu'))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        
        model.add(Dense(classes))
        model.add(Activation('softmax'))
        return model

In [ ]:
model = More_Layer.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

history = model.fit(X_train, y_train, \
                   batch_size=BATCH_SIZE, epochs=NB_EPOCH,\
                   verbose=VERBOSE, validation_split=VALIDATION_SPLIT)
score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
draw(history)

### More data
- 데이터셋의 회전, 크기 조정, 수평 수직 뒤집기, 확대 축소, 채널 이동등의 변형을 통해 데이터를 증가시킬 수 있다

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

NUM_TO_AUGMENT = 5

data_gen = ImageDataGenerator(
    rotation_range=40,\
    width_shift_range=0.2,\
    height_shift_range=0.2, \
    zoom_range=0.2,\
    horizontal_flip=True,\
    fill_mode='nearest'
)

In [ ]:
xtas, ytas = [], []
for i in range(X_train.shape[0]):
    num_aug = 0
    x = X_train[i]
    x = x.reshape((1,) + x.shape)
    
    for x_aug in data_gen.flow(x, batch_size=1, save_to_dir='./cifar_preview',\
                              save_prefix='cifar', save_format='jpeg'):
        if num_aug >= NUM_TO_AUGMENT:
            break
        xtas.append(x_aug[0])
        num_aug += 1

In [ ]:
data_gen.fit(X_train)

In [ ]:
history = model.fit_generator(data_gen.flow(X_train, y_train, batch_size=BATCH_SIZE), \
                              steps_per_epoch=X_train.shape[0], epochs=NB_EPOCH, verbose=VERBOSE)
score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model_json = model.to_json()
open('cifar10_arch.json', 'w').write(model_json)
model.save_weights('cifar10_we.h5', overwrite=True)